In [323]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# loading data
#a = pd.read_excel("3 - changing column names for LLM.xlsx")
#preventing from frequent loading 


In [324]:

df = a
#drop rows with at least 30 missing values
#df.dropna(thresh=df.shape[1] - 10, inplace=True)

# Drop column: 'L1_BloodGroup_First'
df = df.drop(columns=['Blood Group'])
# Drop rows with missing data in column: 'Outcome_InhospitalMortality'
df = df.dropna(subset=['Inhospital Mortality'])


In [325]:
# imputing missing values


# splitting internal and external validation
df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]
df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])
X = df_internal_validation.drop(columns=["Inhospital Mortality","ICU admission","Intubation","Dialysis"])
y  = df_internal_validation['Inhospital Mortality']
X_ex = df_external_validation.drop(columns=["Inhospital Mortality","ICU admission","Intubation","Dialysis"])
y_ex  = df_external_validation['Inhospital Mortality']
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.2 , random_state = 42)
num =  [
    "O2 saturation without supply",
    "pulse rate",
    "diastolic Blood pressure",
    "Systolic Blood pressure",
    "respiratory rate",
    "Temperature",
    "WBC",
    " Lymphocyte count",
    " Neutrophils percentage",
    "Platelets ",
    "Hemoglobin",
    "MCV",
    "CR",
    "sodium",
    "potassium",
    "alkaline phosphatase",
    "ESR",
    "CPK",
    "PTT",
    "PT",
    "Age"
]
cat = X.drop(columns= num).columns.tolist()
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

X_train_cat = imputer.fit_transform(np.array(X_train[cat]))
X_test_cat = imputer.fit_transform(np.array(X_test[cat]))
X_ex_cat = imputer.fit_transform(np.array(X_ex[cat]))

X_train_cat = pd.DataFrame(X_train_cat, columns=cat)
X_test_cat = pd.DataFrame(X_test_cat, columns=cat)
X_ex_cat = pd.DataFrame(X_ex_cat, columns=cat)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=50, random_state=0)

X_train_num = imputer.fit_transform(X_train[num])
X_test_num = imputer.fit_transform(X_test[num])
X_ex_num = imputer.fit_transform(X_ex[num])

X_train_num = pd.DataFrame(X_train_num, columns=num)
X_test_num = pd.DataFrame(X_test_num, columns=num)
X_ex_num = pd.DataFrame(X_ex_num, columns=num)

X_train = pd.concat([X_train_cat, X_train_num], axis=1)
X_test = pd.concat([X_test_cat, X_test_num], axis=1)
X_ex = pd.concat([X_ex_cat, X_ex_num], axis=1)


In [326]:
from sklearn.linear_model import Lasso

def lasso_MO_ICU(X_train, y_train,X_test , X_ex, max_iter=1000):

    
    lasso = Lasso(alpha=0.001,random_state=42 , max_iter=max_iter)
    lasso.fit(X_train, y_train)


    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_ex = pd.DataFrame(X_ex)

    selected_feature_indices = sorted_indices[:40]
    selected_feature_indices = np.where(lasso.coef_ != 0)[0]


    X_train_selected_Mortality_ICU = X_train.iloc[:, selected_feature_indices]
    X_test_selected_Mortality_ICU = X_test.iloc[:, selected_feature_indices]
    X_ex = X_ex.iloc[:, selected_feature_indices]
    return X_train_selected_Mortality_ICU ,X_test_selected_Mortality_ICU,X_ex, selected_feature_indices

X_train,X_test,X_ex, selected_feature_indices = lasso_MO_ICU(X_train, y_train,X_test,X_ex, max_iter=2000)


In [327]:
from imblearn.under_sampling import RandomUnderSampler

under_sampler = RandomUnderSampler()
X_train, y_train = under_sampler.fit_resample(X_train, y_train)
X_test, y_test = under_sampler.fit_resample(X_test, y_test)
X_ex, y_ex = under_sampler.fit_resample(X_ex, y_ex )


In [328]:
for i in [X_train ,y_train, X_test ,y_test, X_ex ,y_ex]:
     i = i.reset_index(drop=True, inplace=True)
    


In [329]:
m = X_ex
z = 'X_ex_u_F'

In [330]:
df = m




# Replace values based on conditions
df['O2 saturation without supply'] = df['O2 saturation without supply'].apply(lambda x: 'normal' if  x >= 95 else 'Lower' )
df['pulse rate'] = df['pulse rate'].apply(lambda x: 'Higher' if  x > 80 else ('Lower' if x <= 60 else 'Normal'))
df['diastolic Blood pressure'] = df['diastolic Blood pressure'].apply(lambda x: 'Higher' if x >= 80 else 'normal')
df['Systolic Blood pressure'] = df['Systolic Blood pressure'].apply(lambda x: 'Higher' if x >= 120 else 'normal')
df['respiratory rate'] = df['respiratory rate'].apply(lambda x: 'Higher' if x >= 18 else ('Lower' if x <= 12 else 'Normal'))
df['Temperature'] = df['Temperature'].apply(lambda x: 'Higher' if x >= 37.2 else ('Lower' if x <= 36.1 else 'Normal'))
df['WBC'] = df['WBC'].apply(lambda x: 'Higher' if x >= 9.5 else ('Lower' if x <= 3.5 else 'Normal'))
df[' Lymphocyte count'] = df[' Lymphocyte count'].apply(lambda x: 'Higher' if x >= 50 else ('Lower' if x <= 20 else 'Normal'))
df[' Neutrophils percentage'] = df[' Neutrophils percentage'].apply(lambda x: 'Higher' if  x >= 75 else ('Lower' if x <= 40 else 'Normal'))
df['Platelets '] = df['Platelets '].apply(lambda x: 'Higher' if  x >= 450 else ('Lower' if x <= 150 else 'Normal'))
df['Hemoglobin'] = df['Hemoglobin'].apply(lambda x: 'Higher' if  x >= 17 else ('Lower' if x <= 13 else 'Normal'))
df['MCV'] = df['MCV'].apply(lambda x: 'Higher' if  x >= 100 else ('Lower' if x <= 80 else 'Normal'))
df['CR'] = df['CR'].apply(lambda x: 'Higher' if  x >= 1.2 else ('Lower' if x <= 0.5 else 'Normal'))
df['sodium'] = df['sodium'].apply(lambda x: 'Higher' if  x >= 145 else ('Lower' if x <= 135 else 'Normal'))
df['potassium'] = df['potassium'].apply(lambda x: 'Higher' if  x >= 5.1 else ('Lower' if x <= 3.5 else 'Normal'))
df['alkaline phosphatase'] = df['alkaline phosphatase'].apply(lambda x: 'Higher' if  x >= 147 else ('Lower' if x <= 44 else 'Normal'))
df['ESR'] = df['ESR'].apply(lambda x: 'Lower' if  x <= 15 else 'Higher' )
df['CPK'] = df['CPK'].apply(lambda x: 'Lower' if  x <= 200 else 'Higher' )
df['PTT'] = df['PTT'].apply(lambda x: 'Higher' if  x >= 40 else ('Lower' if x <= 30 else 'Normal'))
df['PT'] = df['PT'].apply(lambda x: 'Higher' if  x >= 12.5 else ('Lower' if x <= 11 else 'Normal'))





In [331]:
main_xtrain = df

In [332]:
main_xtrain["Gender"] = main_xtrain["Gender"].replace(0 , "female.")
main_xtrain["Gender"] = main_xtrain["Gender"].replace(1 , "male;")
gender = main_xtrain["Gender"]

# Symptomps

In [333]:

symptoms = df[[
'Caugh',
 'Dyspnea',
 'Fever',
 'Chiver',
 'Mylagia',
 'Weakness',
 'loss of consciousness',
 'nausea/Vomit',
 'Diarhhea',
 'ChestPain',
 'Headache',

]]

In [334]:
def convert_row_to_symptoms(row):
    preset_symptoms = []
    for symptom, value in row.items():
        if value == 1:
            preset_symptoms.append(symptom)

    return preset_symptoms


# Iterate through the dataframe rows and print the converted format
main_xtrain['symptomps'] =1 
for index, row in df.iterrows():
    preset = convert_row_to_symptoms(row)
    sentence = ", ".join(preset) 
    
    if sentence ==", ":
        main_xtrain['symptomps'][index] = "No present symptomps. "
    else:
        main_xtrain['symptomps'][index] = f'present symptoms are {sentence}'

C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\39753817.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_xtrain['symptomps'][index] = f'present symptoms are {sentence}'
C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\39753817.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'present symptoms are Caugh, Dyspnea, Fever, Cancer, symptomps' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  main_xtrain['symptomps'][index] = f'present symptoms are {sentence}'


In [335]:
main_xtrain['symptomps']

0      present symptoms are Caugh, Dyspnea, Fever, Ca...
1      present symptoms are Dyspnea, Weakness, hypert...
2      present symptoms are Diarhhea, Curremt Smoker,...
3      present symptoms are Caugh, Dyspnea, hypertens...
4      present symptoms are Dyspnea, Curremt Smoker, ...
                             ...                        
565    present symptoms are Dyspnea, Fever, Mylagia, ...
566    present symptoms are Fever, Chiver, Weakness, ...
567    present symptoms are Caugh, Fever, Chiver, Dia...
568    present symptoms are nausea/Vomit, diabetes me...
569    present symptoms are Caugh, Dyspnea, Fever, Ch...
Name: symptomps, Length: 570, dtype: object

# Past medical history

In [336]:
df = main_xtrain

#Past Medical History

Past =df[['Diarhhea',
 'ChestPain',
 'Headache',
 'Curremt Smoker',
 'hypertension',
 'Congestive Heart Failure',
 'COPD',
 'diabetes mellitus',
 'cerebral vascular accident',
 'Chronic kidney disease',
 'Cancer',
 'Alzhimer'] ]



In [337]:
main_xtrain['Past medical history'] = 1 
for index, row in Past.iterrows():
    preset= convert_row_to_symptoms(row)
    if len(preset) >1 :
      sentence = ", ".join(preset)
      main_xtrain['Past medical history'][index] = f"Past medical history are  {sentence} . "
    elif len(preset) == 1 :
      sentence = ", ".join(preset)
      main_xtrain['Past medical history'][index] = f"Past medical history is  {sentence} . "
    else:
        sentence = ", ".join(preset) 
        main_xtrain['Past medical history'][index] = f"No Past medical history"

C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\872233243.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_xtrain['Past medical history'][index] = f"Past medical history is  {sentence} . "
C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\872233243.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Past medical history is  Cancer . ' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  main_xtrain['Past medical history'][index] = f"Past medical history is  {sentence} . "


In [338]:
main_xtrain['Past medical history']

0                     Past medical history is  Cancer . 
1      Past medical history are  hypertension, diabet...
2      Past medical history are  Diarhhea, Curremt Sm...
3               Past medical history is  hypertension . 
4             Past medical history is  Curremt Smoker . 
                             ...                        
565                 Past medical history is  Headache . 
566        Past medical history is  diabetes mellitus . 
567        Past medical history are  Diarhhea, Cancer . 
568        Past medical history is  diabetes mellitus . 
569      Past medical history are  Diarhhea, Headache . 
Name: Past medical history, Length: 570, dtype: object

In [339]:
main_xtrain = main_xtrain.loc[: , 'symptom to referral':'Past medical history']

In [340]:
main_xtrain['symptom to referral'] = "symptom to referral is " + main_xtrain['symptom to referral'].astype(str)+" days"

# lab test

In [341]:
def convert_row_to_lab(row):
    lab_high = []
    lab_low = []
    lab_normal = []
    for symptom, value in row.items():
        if value == 'Higher':
            lab_high.append(symptom)
        elif value == 'Normal':
            lab_normal.append(symptom)
        else:
            lab_low.append(symptom)
    return lab_high, lab_low , lab_normal

# Iterate through the dataframe rows and print the converted format
lab = df[[
 'O2 saturation without supply',
 'pulse rate',
 'diastolic Blood pressure',
 'Systolic Blood pressure',
 'respiratory rate',
 'Temperature',
 'WBC',
 ' Lymphocyte count',
 ' Neutrophils percentage',
 'Platelets ',
 'Hemoglobin',
 'MCV',
 'CR',
 'sodium',
 'potassium',
 'alkaline phosphatase',
 'ESR',
 'CPK',
 'PTT',
 'PT',
]]

main_xtrain['lab data']=1
for index, row in df.iterrows():
    lab_high, lab_low , lab_normal = convert_row_to_lab(row)
    sentence1 = ", ".join(lab_high)
    sentence2 = ", ".join(lab_low)
    main_xtrain['lab data'][index] = f" {sentence1} are higher than  normal range , {sentence2} are lower than  normal range . "

C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\1819293961.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_xtrain['lab data'][index] = f" {sentence1} are higher than  normal range , {sentence2} are lower than  normal range . "
C:\Users\Mohammad reza\AppData\Local\Temp\ipykernel_1500\1819293961.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' pulse rate, Systolic Blood pressure, respiratory rate, Temperature,  Neutrophils percentage, CR, alkaline phosphatase, ESR, PT are higher than  normal range , Gender, Caugh, Dyspnea, Fever, Chiver, Mylagia, Weakness, loss of consciousness, nausea/Vomit, Diarhhea, ChestPain, Headache, Curremt Smoker, hypertension, Congestive Heart Failure, COPD, diabetes mellitus, cerebra

In [342]:
main_xtrain['lab data']

0       pulse rate, Systolic Blood pressure, respirat...
1       pulse rate, diastolic Blood pressure, Systoli...
2       pulse rate, respiratory rate, Temperature, WB...
3       Systolic Blood pressure,  Neutrophils percent...
4       pulse rate, respiratory rate, WBC,  Lymphocyt...
                             ...                        
565     pulse rate, respiratory rate, alkaline phosph...
566     diastolic Blood pressure, Systolic Blood pres...
567     pulse rate, diastolic Blood pressure, respira...
568     pulse rate, Temperature,  Neutrophils percent...
569     pulse rate, diastolic Blood pressure, respira...
Name: lab data, Length: 570, dtype: object

# Age

In [343]:
main_xtrain["Age"]= "age is "+main_xtrain["Age"].astype(str)+". "

# mortality

In [344]:
y_train = pd.DataFrame(y_train)

In [345]:
pd.DataFrame(y_ex)

,Inhospital Mortality
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
565,1.0
566,1.0
567,1.0
568,1.0


In [346]:
def rename_mortality(data):
    mortality = []
    for i in data:
        if i == 1.0:
            mortality.append('survive')
        else:
            mortality.append('die')
    return mortality

In [347]:
y_ex = pd.DataFrame(y_ex)
y_test= pd.DataFrame(y_test)

In [348]:

y_train.loc[: ,"Inhospital Mortality"] = rename_mortality(y_train.loc[: ,"Inhospital Mortality"])
y_test.loc[: ,"Inhospital Mortality"] = rename_mortality(y_test.loc[: ,"Inhospital Mortality"])
y_ex.loc[: ,"Inhospital Mortality"] = rename_mortality(y_ex.loc[: ,"Inhospital Mortality"])

In [349]:
y_train

,Inhospital Mortality
0,die
1,die
2,die
3,die
4,die
...,...
2471,survive
2472,survive
2473,survive
2474,survive


In [350]:
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
y_ex.reset_index(drop=True, inplace=True)

# y_train_mortality

In [351]:
y_train_mortality = y_ex

In [352]:
main_xtrain= pd.concat([main_xtrain , y_train_mortality], axis=1)

In [353]:
gender = pd.DataFrame(gender)

In [354]:
main_xtrain = pd.concat([main_xtrain,gender],axis=1)

In [355]:
main_xtrain["patient medical hidtory"] = main_xtrain["Age"]+" "+main_xtrain["Gender"].astype('string') +" "+main_xtrain["symptomps"]+" "+ main_xtrain["Past medical history"]+" "+main_xtrain['lab data']

In [356]:
final_train_dataset = pd.concat([main_xtrain["patient medical hidtory"], y_train_mortality] , axis=1)

In [357]:
final_train_dataset

,patient medical hidtory,Inhospital Mortality
0,age is 75.0. male; present symptoms are Caugh...,die
1,age is 68.0. female. present symptoms are Dys...,die
2,age is 60.0. female. present symptoms are Dia...,die
3,age is 85.0. male; present symptoms are Caugh...,die
4,age is 39.0. male; present symptoms are Dyspn...,die
...,...,...
565,age is 26.0. female. present symptoms are Dys...,survive
566,age is 65.0. male; present symptoms are Fever...,survive
567,age is 37.0. male; present symptoms are Caugh...,survive
568,age is 69.0. male; present symptoms are nause...,survive


In [358]:
# saving the model

final_train_dataset.to_excel(f"{z}.xlsx", index= False)